In [1]:
from dataclasses import dataclass
from pathlib import Path
import os

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [2]:
os.chdir('../')

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Check if artifacts_root is accessible
        if 'artifacts_root' in self.config:
            # Access 'artifacts_root' from config
            artifacts_root_path = Path(self.config['artifacts_root'])
            full_artifacts_path = CONFIG_FILE_PATH.parent / artifacts_root_path
            print(full_artifacts_path)

            # Ensure the directory exists
            full_artifacts_path.mkdir(parents=True, exist_ok=True)

            # Pass the full path to create_directories
            create_directories([full_artifacts_path])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
  

In [5]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_URL, filename = self.config.local_data_file)
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    

    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extract the zip file to the directory
        Function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-22 22:01:55,851] {d:\development\machine learning\projects\text-summerization-nlp\src\textSummarizer\utils\common.py:29} INFO - YAML file read successfully: config\config.yaml
[2024-06-22 22:01:55,855] {d:\development\machine learning\projects\text-summerization-nlp\src\textSummarizer\utils\common.py:29} INFO - YAML file read successfully: params.yaml
Artifacts root path: artifacts


TypeError: isinstance() argument 2 cannot be a parameterized generic

In [ ]:
from textSummarizer.utils.common import read_yaml

config = read_yaml(CONFIG_FILE_PATH)
print(config)  # Check if this prints the expected content of config.yaml

[2024-06-22 21:59:33,957] {d:\development\machine learning\projects\text-summerization-nlp\src\textSummarizer\utils\common.py:29} INFO - YAML file read successfully: config\config.yaml
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/DhruvGandhi31/Text-Summerization-NLP/raw/main/data/samsumdata.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
